<a href="https://colab.research.google.com/github/AntoninaRemizova/ML/blob/main/ML_class_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy requests
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 56.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Импорт библиотек

In [2]:
import spacy
import random
import requests

In [3]:
nlp = spacy.load("en_core_web_md")

api_key = "e78b201d"

# информация о фильме

In [18]:
def get_movie_info(movie_title, api_key):
    api_url = f"http://www.omdbapi.com/?t={movie_title}&apikey={api_key}"
    try:
        response = requests.get(api_url)
        response_dict = response.json()
        if response.status_code == 200:
            if response_dict.get("Response") == "True":
                movie_data = {
                    "title": response_dict.get("Title", "N/A"),
                    "year": response_dict.get("Year", "N/A"),
                    "genre": response_dict.get("Genre", "N/A"),
                    "director": response_dict.get("Director", "N/A"),
                    "actors": response_dict.get("Actors", "N/A"),
                    "plot": response_dict.get("Plot", "N/A"),
                    "rating": response_dict.get("imdbRating", "N/A"),
                    "runtime": response_dict.get("Runtime", "N/A")
                }
                return movie_data
            else:
                print(f"Ошибка: {response_dict.get('Error', 'Unknown error')}")
                return None
        else:
            print(f"Ошибка: {response.status_code}")
            return None

    except Exception as e:
        print(f"Ошибка: {e}")
        return None

# бот

In [22]:
def chatbot(statement):
    movie_info_questions = [
        nlp("Tell me about the movie"),
        nlp("What do you know about the film"),
        nlp("Information about the film"),
        nlp("I want to know about movie")
    ]
    movie_rating_questions = [
        nlp("What is the rating of"),
        nlp("What is the rating for"),
        nlp("How is rated"),
        nlp("IMDb rating for"),
        nlp("Score for movie")
    ]
    movie_year_questions = [
        nlp("When was released"),
        nlp("Release year of"),
        nlp("What year is movie"),
        nlp("Year of the film"),
        nlp("When the film came out"),
        nlp("In which year"),
        nlp("What year did come out"),
    ]
    movie_genre_questions = [
        nlp("What genre is"),
        nlp("What kind of film is"),
        nlp("What type of movie is"),
        nlp("What is the genre of")
    ]
    movie_director_questions = [
        nlp("Who directed"),
        nlp("Who made the film"),
        nlp("Director of film"),
        nlp("Who is the director of")
    ]

    statement = nlp(statement)

    similarity_scores = {
        "info": max([question.similarity(statement) for question in movie_info_questions]),
        "rating": max([question.similarity(statement) for question in movie_rating_questions]),
        "year": max([question.similarity(statement) for question in movie_year_questions]),
        "genre": max([question.similarity(statement) for question in movie_genre_questions]),
        "director": max([question.similarity(statement) for question in movie_director_questions])
    }

    min_similarity = 0.7

    max_score = max(similarity_scores.values())
    question_type = max(similarity_scores, key=similarity_scores.get)

    if max_score >= min_similarity:
        movie_title = ""

        for ent in statement.ents:
            if ent.label_ in ["PERSON", "ORG", "WORK_OF_ART"]:
                movie_title = ent.text
                break

        if not movie_title:
            text = statement.text.lower()
            keywords = ["about", "movie", "film", "of", "for", "is"]

            for i, word in enumerate(statement):
                if word.text.lower() in keywords and i + 1 < len(statement):
                    movie_title = " ".join([token.text for token in statement[i+1:]])
                    break

        if movie_title:
            movie_data = get_movie_info(movie_title, api_key)

            if movie_data:
                if question_type == "info":
                    response_templates = [
                        f"Here's what I know about '{movie_data['title']}': ",
                        f"Information about '{movie_data['title']}': ",
                        f"I found this about '{movie_data['title']}': "
                    ]

                    info = (f"\n     It was released in {movie_data['year']}. "
                           f"\n     Genre: {movie_data['genre']}. "
                           f"\n     Directed by {movie_data['director']}. "
                           f"\n     Actors: {movie_data['actors']}. "
                           f"\n     Plot: {movie_data['plot'][:150]}... "
                           f"\n     IMDb rating: {movie_data['rating']}/10. "
                           f"\n     Runtime: {movie_data['runtime']}.")

                    return random.choice(response_templates) + info

                elif question_type == "rating":
                    response_templates = [
                        f"The IMDb rating for '{movie_data['title']}' is {movie_data['rating']}/10.",
                        f"'{movie_data['title']}' has a rating of {movie_data['rating']} on IMDb.",
                    ]
                    return random.choice(response_templates)

                elif question_type == "year":
                    response_templates = [
                        f"'{movie_data['title']}' was released in {movie_data['year']}.",
                        f"The film '{movie_data['title']}' came out in {movie_data['year']}.",
                    ]
                    return random.choice(response_templates)

                elif question_type == "genre":
                    response_templates = [
                        f"'{movie_data['title']}' is a {movie_data['genre']} movie.",
                        f"The genre of '{movie_data['title']}' is {movie_data['genre']}.",
                    ]
                    return random.choice(response_templates)

                elif question_type == "director":
                    response_templates = [
                        f"'{movie_data['title']}' was directed by {movie_data['director']}.",
                        f"The director of '{movie_data['title']}' is {movie_data['director']}.",
                    ]
                    return random.choice(response_templates)
            else:
                return f"Sorry, I couldn't find information about '{movie_title}'."

    error_responses = [
        "I'm sorry, I don't understand. Try asking about movies, for example: 'Tell me about The Matrix'",
        "Try asking something like: 'What is the rating of Inception?' or 'What genre is Titanic?'"
    ]
    return random.choice(error_responses)

In [6]:
def chat():
    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye', 'goodbye']:
            print("Bot: Have a great day!")
            break

        if user_input.lower() in ['hello', 'hi']:
            print("Bot: Hello! You can ask me anything about movies, actors, ratings and more.")

        elif user_input.lower() in ['thank you', 'thanks']:
            print("Bot: You're welcome!")

        elif user_input:
            response = chatbot(user_input)
            print(f"Bot: {response}")

# чат

*   What is the rating for The Matrix?
*   Tell me about Forrest Gump
*   Who created Interstellar?
*   What genre is Titanic?
*   When did Grinch come out?


In [23]:
chat()


You: Tell me about Forrest Gump
Bot: Here's what I know about 'Forrest Gump': 
     It was released in 1994. 
     Genre: Drama, Romance. 
     Directed by Robert Zemeckis. 
     Actors: Tom Hanks, Robin Wright, Gary Sinise. 
     Plot: The history of the United States from the 1950s to the '70s unfolds from the perspective of an Alabama man with an IQ of 75, who yearns to be reunited... 
     IMDb rating: 8.8/10. 
     Runtime: 142 min.

You: When did Grinch come out?
Bot: 'Grinch' was released in 2018.

You: What genre is Titanic?
Bot: 'Titanic' is a Drama, Romance movie.

You: Who created Interstellar?
Bot: The director of 'Interstellar' is Christopher Nolan.

You: What is the rating for The Matrix?
Bot: The IMDb rating for 'The Matrix' is 8.7/10.

You: bye
Bot: Have a great day!
